In [ ]:
rm(list=ls())
my.libs=c("Seurat","scran","scater","harmony","ggtree","ggplot2","cowplot","pROC","RColorBrewer","org.Hs.eg.db")
lapply(my.libs, require, character.only = TRUE)

In [ ]:
dir=c("/home/matthias/chicken/")
dat.dir=c("/home/matthias/chicken/source_data/")
load(paste0(dat.dir,"QC_ok_mtRNA30_LEG_1.RData"))
leg1=dat1
load(paste0(dat.dir,"QC_ok_mtRNA30LEG_2.RData"))
leg2=dat1
load(paste0(dat.dir,"QC_ok_mtRNA30_LEG_3.RData"))
leg3=dat1
leg1@meta.data$replicate=rep(1,nrow(leg1@meta.data))
leg2@meta.data$replicate=rep(2,nrow(leg2@meta.data))
leg3@meta.data$replicate=rep(3,nrow(leg3@meta.data))

In [ ]:
chicken.comb = merge(leg1, y = leg2, project = "chicken")
chicken.comb1 =merge(chicken.comb,y=leg3,project = "chicken")

In [ ]:
head(chicken.comb1@meta.data)

In [ ]:
chicken.comb1=SCTransform(chicken.comb1, variable.features.n = NULL, vars.to.regress = c("percent.mt","CC.Difference"), 
            conserve.memory = F, return.only.var.genes = F) %>% RunPCA(verbose = TRUE)
chicken.comb1@meta.data$replicate=as.factor(chicken.comb1@meta.data$replicate)
all.vars=as.character(rownames(chicken.comb1))
dat1_harmony = RunHarmony(chicken.comb1, assay.use="SCT",group.by.vars = "replicate",vars_use=all.vars) # assay.use="SCT #vars_use=all.vars

In [ ]:
Seurat::DimHeatmap(dat1_harmony, reduction = "harmony",dims = 1, cells = 500, balanced = TRUE)  ### plot the top 500 cells (highest scores)
Seurat::DimHeatmap(dat1_harmony, dims = 2,reduction = "harmony", cells = 500, balanced = TRUE)  ### plot the top 500 cells (highest scores)
Seurat::DimHeatmap(dat1_harmony, dims = 3,reduction = "harmony", cells = 500, balanced = TRUE)
Seurat::DimHeatmap(dat1_harmony, dims = 4, reduction = "harmony",cells = 500, balanced = TRUE)
Seurat::DimHeatmap(dat1_harmony, dims = 5,reduction = "harmony", cells = 500, balanced = TRUE)
Seurat::DimHeatmap(dat1_harmony, dims = 6,reduction = "harmony", cells = 500, balanced = TRUE)
Seurat::ElbowPlot(dat1_harmony,reduction="harmony")

In [ ]:
sig_PCs=c(1:16)
sce.seurat <- RunUMAP(dat1_harmony, reduction = "harmony",dims = sig_PCs)

In [ ]:
#sig_PCs=c(1,2,3,4,5,6,9,10,11,12,13,14,15)
sig_PCs=c(1:16)
sce.seurat1=Seurat::FindNeighbors(sce.seurat,reduction = "harmony",k.param = 20,dims=sig_PCs)  

In [ ]:
clust_test=function(sce) {
  cont=as.data.frame(matrix(NA,nrow=length(unique(Idents(sce))), ncol=4))
  colnames(cont)=c("cluster","cells","mean_total_UMI","mean_features")
  i=1
  for (k in unique(Idents(sce))) {
    print(c(k))
    cont[i,1]=c(k)
    dat23=sce@assays$RNA@counts[,Idents(sce)%in%c(k)]
    cont[i,2]=table(Idents(sce)%in%c(k))[2]
    cont[i,3]=mean(apply(dat23,2,function(x)sum(x)))
    cont[i,4]=mean(apply(dat23,2,function(x)table(x %in% c(0))[1]))
    i=i+1
  }
  cont=cont[order(as.numeric(as.character(cont$cluster))),]
  return(cont)
}

In [ ]:
library(ggplot2)
library(ggtree)
library(cowplot)
clust.res=0.2
sce.seurat1=Seurat::FindClusters(sce.seurat1,resolution=clust.res)  
sce.seurat1= Seurat::BuildClusterTree(sce.seurat1,dims=sig_PCs)
#sce.seurat1 <- RunUMAP(sce.seurat1, dims = sig_PCs)
tree1=sce.seurat1@tools$`Seurat::BuildClusterTree`
pl=Seurat::DimPlot(sce.seurat1, reduction = "umap")
pl= pl+ ggtitle(paste("UMAP and phylo-tree visualization of cell clusters \n at resolution",clust.res,sep=" ")) +
  theme (plot.title = element_text(color="black", size=14, face="bold",hjust = 0.5))
pl.matrix=ggplot_build(pl)$data[1][[1]]
uni_pl_mat=pl.matrix[!duplicated(pl.matrix$group),]
tr=ggtree(tree1,layout="circular") + geom_tiplab(aes(angle=angle), lwd=5,offset=.5,fontface="bold")  
plot_grid(pl, tr, align = "v", nrow = 2, rel_heights = c(2/3, 1/3),axis="l")    
#jpeg(paste0(out,"CLUSTERING_NO_PBMC.jpeg"))
 #plot_grid(pl, tr, align = "v", nrow = 2, rel_heights = c(2/3, 1/3),axis="l")
#dev.off()
res1=clust_test(sce.seurat1)
res1
#write.csv(res1,file=paste0(out,"CLUSTERING_NO_PBMC.csv"))

In [ ]:
save(sce.seurat1,file="all_chicken_legs_harmony_integrated_mtRNA30.RData")

In [ ]:
DefaultAssay(sce.seurat1)=c("RNA")
i=c("Krt9LC2")  #LOR1-EDC #Krt5L1 Krt14L1  Krt9LC1 CBP63-K  Krt9L4    Krt9LC2 == cluster 4   Krt9L4 == cluster 6
p=FeaturePlot(sce.seurat1,features = as.character(i),min.cutoff=0.1,max.cutoff=5)
p

In [ ]:
head(sce.seurat1@meta.data)

# check saved objects

In [ ]:
library(Seurat)
jpeg("all_chicken_legs_harmony_mt30.RData")
UMAPPlot(sce.seurat1,split.by="replicate")
dev.off()
UMAPPlot(sce.seurat1,split.by="replicate")

In [ ]:
head(sce.seurat1@meta.data)

In [ ]:
phe=sce.seurat@meta.data[,c("percent.mt","nCount_RNA","nFeature_RNA","doub_score","nCount_SCT","nFeature_SCT","S.Score","replicate")]
phe$ID=as.character(rownames(sce.seurat1@meta.data))
pcs=sce.seurat1@reductions$harmony@cell.embeddings[,1:15]
reg.dat=as.data.frame(cbind(phe,pcs))



In [ ]:
hist(reg.dat$percent.mt)

In [ ]:
head (reg.dat$percent.mt)

In [ ]:
summary(lm(paste("harmony_",k,"~percent.mt",sep=""),data=reg.dat))


In [ ]:
sig_PCs=c(15)
mt_vs_PCs=data.frame(ID= rep(NA,sig_PCs),Estimate=rep(NA,sig_PCs),std_err=rep(NA,sig_PCs),t_val=rep(NA,sig_PCs),P_val=rep(NA,sig_PCs))

for (k in 1:sig_PCs){
  fit=summary(lm(paste("harmony_",k,"~percent.mt",sep=""),data=reg.dat))$coefficients[2,]
  mt_vs_PCs[k,1]=paste("harmony_",k,sep="")
  mt_vs_PCs[k,2:5]=fit
}
print(paste("multiple testing threshold is ",0.05/sig_PCs,sep=""))
print("++++++++++ associations between mitochnodrial RNA content and PCs in analysis +++++++++++++")

In [ ]:
# define the number of PCs you think make sense in your analysis
sig_PCs=c(15)

mt_vs_PCs=data.frame(ID= rep(NA,sig_PCs),Estimate=rep(NA,sig_PCs),std_err=rep(NA,sig_PCs),t_val=rep(NA,sig_PCs),P_val=rep(NA,sig_PCs))
for (k in 1:sig_PCs){
  fit=summary(lm(paste("PC_",k,"~perc_mt",sep=""),data=reg.dat))$coefficients[2,]
  mt_vs_PCs[k,1]=paste("PC_",k,sep="")
  mt_vs_PCs[k,2:5]=fit
}
print(paste("multiple testing threshold is ",0.05/sig_PCs,sep=""))
print("++++++++++ associations between mitochnodrial RNA content and PCs in analysis +++++++++++++")
mt_vs_PCs
## library size 
libS_vs_PCs=data.frame(ID= rep(NA,sig_PCs),Estimate=rep(NA,sig_PCs),std_err=rep(NA,sig_PCs),t_val=rep(NA,sig_PCs),P_val=rep(NA,sig_PCs))
for (k in 1:sig_PCs){
  fit=summary(lm(paste("PC_",k,"~log_LIBsize",sep=""),data=reg.dat))$coefficients[2,]
  libS_vs_PCs[k,1]=paste("PC_",k,sep="")
  libS_vs_PCs[k,2:5]=fit
}
print("++++++++++ associations between library size and PCs in analysis +++++++++++++")
libS_vs_PCs
overview["PCs_assocTECH",2]=(length(which(libS_vs_PCs$P_val < (0.05/sig_PCs))) + length(which(mt_vs_PCs$P_val < (0.05/sig_PCs))))